In [7]:
import requests
import json

In [8]:
match_id = 2812759987

In [9]:
host = "https://api.opendota.com/api/matches/" + str(match_id)
data = {'match_id': match_id}

In [10]:
request = requests.get(host, data=data)

In [11]:
request.status_code

200

In [51]:
match_data = request.content
match_data = json.loads(match_data)

In [52]:
match_data.keys()

[u'replay_url',
 u'barracks_status_dire',
 u'match_id',
 u'barracks_status_radiant',
 u'cluster',
 u'replay_salt',
 u'first_blood_time',
 u'chat',
 u'game_mode',
 u'duration',
 u'radiant_win',
 u'lobby_type',
 u'tower_status_dire',
 u'teamfights',
 u'series_type',
 u'objectives',
 u'all_word_counts',
 u'version',
 u'cosmetics',
 u'leagueid',
 u'stomp',
 u'engine',
 u'human_players',
 u'start_time',
 u'match_seq_num',
 u'series_id',
 u'tower_status_radiant',
 u'negative_votes',
 u'comeback',
 u'picks_bans',
 u'positive_votes',
 u'my_word_counts',
 u'radiant_xp_adv',
 u'region',
 u'patch',
 u'radiant_gold_adv',
 u'players']

In [26]:
# Here we analyze a player
player1 = match_data['players'][0]

In [29]:
player1.keys()

[u'purchase_time',
 u'lane_efficiency',
 u'gold',
 u'sentry_kills',
 u'lane_efficiency_pct',
 u'actions',
 u'damage_inflictor_received',
 u'lh_t',
 u'damage_taken',
 u'kill_streaks',
 u'cosmetics',
 u'hero_id',
 u'necronomicon_kills',
 u'kills_log',
 u'sentry_uses',
 u'account_id',
 u'kills',
 u'kills_per_min',
 u'start_time',
 u'isRadiant',
 u'purchase_ward_sentry',
 u'leaver_status',
 u'killed',
 u'stuns',
 u'gold_per_min',
 u'name',
 u'level',
 u'actions_per_min',
 u'damage_inflictor',
 u'life_state_dead',
 u'patch',
 u'item_4',
 u'item_5',
 u'item_2',
 u'item_3',
 u'item_0',
 u'lose',
 u'sen_left_log',
 u'buyback_count',
 u'obs_placed',
 u'match_id',
 u'kda',
 u'pings',
 u'gold_reasons',
 u'total_gold',
 u'item_uses',
 u'duration',
 u'sen',
 u'lobby_type',
 u'denies',
 u'ancient_kills',
 u'killed_by',
 u'neutral_kills',
 u'permanent_buffs',
 u'last_login',
 u'gold_t',
 u'xp_per_min',
 u'ability_uses',
 u'item_1',
 u'abandons',
 u'rune_pickups',
 u'assists',
 u'obs_left_log',
 u'xp_

https://github.com/kronusme/dota2-api/blob/master/data/heroes.json
https://github.com/kronusme/dota2-api/blob/master/data/items.json

Of interest from the player fields are:
    1. gold
    2. last_hints
    3. damage_taken
    4. isRadiant
    5. hero_id
    6. actions_per_min
    7. kills
    8. kills_per_min
    9. item_win for end state items
    10. kda
    11. assists
    12. deaths
    13. total_gold and total_xp
    14. denies
    15. item_0, item_1, item_2, ..., item_5
    16. lane_role
    17. lane_efficiency_pct
    18. solo_competitive_rank

In [50]:
player1['lane_efficiency_pct']

55

In [53]:
match_data['radiant_win']

False